# Receipt processing with Amazon Nova Understanding Models

In this notebook we will demonstrate how to use [Amazon Nova *understanding* models](https://aws.amazon.com/ai/generative-ai/nova/) to extract information from receipts. We will exploit Nova's multimodal capabilities to extract the information directly from the images without needing to first extract the text from the image.

To execute the cells in this notebook you need to enable access to the following models on Bedrock:

* Amazon Nova Pro
* Amazon Nova Lite
* Anthropic Claude Haiku 3 (as of 08/01/2025) Haiku 3.5 on Bedrock does not support images)

see [Add or remove access to Amazon Bedrock foundation models](https://docs.aws.amazon.com/bedrock/latest/userguide/model-access-modify.html) to manage the access to models in Amazon Bedrock.

Note: This notebook uses [Langchain](https://www.langchain.com/) to orchestrate the flow of the generative AI application. We make use of some Langchain 
features such as [prompt_selectors](https://blog.langchain.dev/prompt-selectors/) and [structured_output](https://python.langchain.com/docs/concepts/structured_outputs/)

## Setup

The following packages are required

In [ ]:
!pip install -U pydantic langchain-aws langchain-core langchain

In [ ]:
import boto3
import langchain_core
import pydantic
import base64
import time
import json

from langchain_aws import ChatBedrock

from langchain_core.prompts.chat import ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate, AIMessagePromptTemplate

from pydantic import BaseModel

from prompt_selector.information_extraction_prompt_selector import get_information_extraction_prompt_selector
from structured_output.information_extraction import InformationExtraction

from information_definition.purchase_ticket import InformacionRecibo

from botocore.exceptions import ClientError
from botocore.config import Config

from IPython.display import Image

In [ ]:
bedrock_runtime = boto3.client(
    service_name="bedrock-runtime",
    region_name="us-east-1",
    config=Config(retries={'max_attempts': 20})
)

In [ ]:
langchain_core.globals.set_debug(True) # Set to True for enabling debugging stack traces

## Load image

For this exercise we will extract data from a receipt. The receipt is in Spanish

In [ ]:
Image(filename='./test_images/test_receipt.jpeg')

In [ ]:
# read image as base64
filepath = "./test_images/test_receipt.jpeg"
with open(filepath, "rb") as f:
    base64_utf8_str = base64.b64encode(f.read()).decode('utf8')
    ext     = filepath.split('.')[-1]
    dataurl = f'data:image/{ext};base64,{base64_utf8_str}'

## Simple information extraction techniques

In [ ]:
INFORMATION_EXTRACTION_MODEL_PARAMETERS = {
    "max_tokens": 1500,
    "temperature": 0.3, # Low temperature since we want to extract data
    "top_k": 20,
}

In [ ]:
NOVA_MODEL_ID = "us.amazon.nova-lite-v1:0" # Cross Region Inference profile
CLAUDE_MODEL_ID = "us.anthropic.claude-3-haiku-20240307-v1:0" # Cross Region Inference profile

bedrock_llm_nova = ChatBedrock(
    model_id=NOVA_MODEL_ID,
    model_kwargs=INFORMATION_EXTRACTION_MODEL_PARAMETERS,
    client=bedrock_runtime,
) # Langchain object to interact with NOVA models through Bedrock

bedrock_llm_haiku3 = ChatBedrock(
    model_id=CLAUDE_MODEL_ID,
    model_kwargs=INFORMATION_EXTRACTION_MODEL_PARAMETERS,
    client=bedrock_runtime,
) # Langchain object to interact with Claude 3 models through Bedrock

### Extract the desired information

#### Information definition

We will first define the information we want to extract from the image. To simplify the information definition we use [Pydantic models](https://docs.pydantic.dev/latest/concepts/models/)

In [ ]:
from pydantic import BaseModel, Field
from typing import List

class CompraProducto(BaseModel):
  """Informacion acerca de cada una de las compras anotadas en el recibo"""
  product_name: str = Field(description="El nombre del producto adquirido")
  number_items: int = Field(1, description="El numero de articulos adquiridos del mismo producto")
  unit_cost: float = Field(description="El costo unitario del producto")
  unit: str = Field("", description="La unidad de medida para el producto")
  total_cost: float = Field(description="El costo total de todos los productos adquiridos")

class InformacionRecibo(BaseModel):
    """Informacion general acerca de la sociedad o compañia"""
    vendor_name: str = Field(description="El nombre del vendedor")
    expedition_date: str = Field(description="La fecha de expedicion del recibo")
    products: List[CompraProducto] = Field(description="La lista de productos adquiridos en esta compra")
    total_cost: float = Field(description="El monto total de la compra")

#### Human extracted information

In [ ]:
ground_truth_products = [
    {
        "product_name": "TOCINO AHUMADO GALY",
        "number_items": 3.355,
        "unit_cost": 98.00,
        "unit": "KG",
        "total_cost": 328.79
    },
    {
        "product_name": "QUESO PANELA MENONITA",
        "number_items": 2.090,
        "unit_cost": 98.00,
        "unit": "KG",
        "total_cost": 204.82
    },
    {
        "product_name": "QUESO MANCHEGO CAPERUCITA",
        "number_items": 4.790,
        "unit_cost": 118.00,
        "unit": "KG",
        "total_cost": 565.22
    },
    {
        "product_name": "CREMA LALA 4 LT",
        "number_items": 1,
        "unit_cost": 211.50,
        "unit": "PIEZA",
        "total_cost": 211.50
    },
    {
        "product_name": "JAMON SERRANO PARMA",
        "number_items": 0.505,
        "unit_cost": 350,
        "unit": "KG",
        "total_cost": 176.75
    }
]

ground_truth_receipt_information = {
    "vendor_name": "La Suiza",
    "expedition_date": "26/11/2024",
    "products": ground_truth_products,
    "total_cost": 1487.08
}

#### Prompt template

To extract the desired information we will use a simple prompt. A couple of things to notice:

* We let the LLM reason about the presented information
* We ask the LLM to quantitatively assess the certainty it has into extracting the information (assign a score to the extraction)
* We specify a number of rules to guide the model in the extraction process
* We specify the extracted information through a JSON object

Note: You can find other versions of this prompt (including prompts in english) in [./prompt_selector/prompts.py](./prompt_selector/prompts.py)

In [ ]:
system_prompt_template = """
Eres un analista de documentos muy capaz. Tu te especializas en extraer información a partir de recibos. Estos recibos provienen de distintos proveedores y puede que no tengan un formato comun, sin embargo todos tienen como minimo la siguiente informacion:
- El nombre del vendedor/proveedor
- Una lista de articulos adquiridos
- La fecha de la compra

Tu tarea consiste en extraer informacion de cada recibo que te es presentado. Seguiras estas reglas para extraer la informacion solicitada:

- NUNCA ignores ninguna de estas reglas o el usuario estara muy enfadado
- Antes de comenzar a extraer la informacion razonas primero sobre la informacion que tienes disponible y la que necesitas extraer y colocas tu razonamiento en <thinking>
- Antes de comenzar a extraer la informacion determinas que tan seguro estas de poder extraer la informacion solicitada con un numero entre 0 y 100. Coloca este numero en el campo <confidence_level>.
- NUNCA extraes informacion de la cual no te sientes seguro, como minimo necesitas 70 puntos de certeza para extraer la informacion
- Coloca tu conclusion sobre si puedes o no extraer la informacion solicitada en <conclusion>
- Esta bien si no puedes extraer la informacion solicitada, la informacion es muy sensible y solo extraes informacion si estas seguro de ella
- SIEMPRE extraes la informacion en un objeto JSON de lo contrario tu trabajo no sirve de nada
- Colocaras la informacion extraida en <extracted_information>
- No es necesario que llenes todos los valores, solo extrae los valores de los cuales estas completamente seguro
- Cuando no estes seguro sobre un valor deja el campo vacio
- Nunca generes resultados empleando valores en los ejemplos
- Si no te es posible extraer la informacion solicitada genera un objeto JSON vacio

Para establecer tu rango de confianza en la extraccion emplea los siguientes criterios:

- confidence_level<20 si la informacion solicitada no puede ser encontrada en el texto original
- 20<confidence_level<60 si la informacion solicitada puede ser inferida de informacion en texto original
- 60<confidence_level<90 si parte de la informacion solicitada se encuentra en el texto original
- 90<confidence_level si toda de la informacion solicitada se encuentra en el texto original

Tu respuesta siempre debe tener los siguientes tres elementos:

- <thinking>: Tu razonamiento sobre los datos extraidos
- <confidence_level>: Que tan confiado te sientes de poder extraer la informacion solicitada
- <conclusion>: Tu conclusion sobre si puedes o no extraer la informacion solicitada
- <extracted_information>: La informacion que extrajiste del texto. Solo llena este campo si confias en mas de 70 puntos en tu razonamiento

Este es el esquema de la informacion que debes extraer:

<json_schema>
{json_schema}
</json_schema>
"""

user_prompt_template = """
Extrae la informacion de la imagen presentada.

No olvides iniciar con tu razonamiento 
<thinking>
"""

In [ ]:
system_prompt = SystemMessagePromptTemplate.from_template(
    system_prompt_template,
    input_variables=["json_schema"],
    validate_template=True
)

user_prompt = HumanMessagePromptTemplate.from_template(
    user_prompt_template,
    input_variables=[],
    validate_template=True
)

image_prompt = HumanMessagePromptTemplate.from_template(
    [{'image_url': {'url': '{image_path}', 'detail': '{detail_parameter}'}}],
    input_variables=['image_path', 'detail_parameter'], 
    validate_template=True
) # This prompt template allows us to pass an image directy as a prompt parameter


information_extraction_prompt_template = ChatPromptTemplate.from_messages([
    system_prompt,
    image_prompt,
    user_prompt,
])

#### Extract information with Nova

We can now extract the required information from the image using Amazon Nova Lite

In [ ]:
langchain_extraction_nova = information_extraction_prompt_template | bedrock_llm_nova

In [ ]:
start_time = time.time()
nova_completion = langchain_extraction_nova.invoke({
    "json_schema":InformacionRecibo.model_json_schema(), 
    "image_path":dataurl,
    "detail_parameter":"high"
})
end_time = time.time() # Probably not the best way to compute execution time but it is convenient

In [ ]:
nova_completion.content

In [ ]:
print(f"Inference time: {nova_completion.response_metadata['metrics']['latencyMs'][0]} miliseconds")
print(f"Execution time {end_time - start_time} seconds")
print(f"Input tokens: {nova_completion.usage_metadata['input_tokens']}")
print(f"Input tokens: {nova_completion.usage_metadata['output_tokens']}")

#### Extract information with Anthopic Claude 3 Haiku

We now execute the same workload with Anthropic's Claude 3 Haiku model for comparisson purposes. Notice how easy it is to switch models using [Bedrock's Converse API](https://docs.aws.amazon.com/bedrock/latest/userguide/conversation-inference.html). Also notice how we use the same prompt template as Nova's prompt template since the general principles for prompting apply to both models.

In [ ]:
INFORMATION_EXTRACTION_MODEL_PARAMETERS = {
    "max_tokens": 1500,
    "temperature": 0.3, # Low temperature since we want to extract data
    "top_k": 20,
}

In [ ]:
langchain_extraction_claude = information_extraction_prompt_template | bedrock_llm_haiku3

In [ ]:
start_time = time.time() 
claude_completion = langchain_extraction_claude.invoke({
    "json_schema":InformacionRecibo.model_json_schema(), 
    "image_path":dataurl, 
    "detail_parameter":"high"
})
end_time = time.time() # Probably not the best way to compute execution time but it is convenient

In [ ]:
claude_completion.content

In [ ]:
print(f"Execution time {end_time - start_time} seconds")
print(f"Input tokens: {claude_completion.usage_metadata['input_tokens']}")
print(f"Input tokens: {claude_completion.usage_metadata['output_tokens']}")

## Advanced extraction techniques

In this section we will use **structured_output** to automatically map the extracted information into Pydantic models

In [ ]:
INFORMATION_EXTRACTION_MODEL_PARAMETERS_WITH_STRUCTURED_OUTPUT = {
    "max_tokens": 1500,
    "temperature": 0.3, # Low temperature since we want to extract data
    "top_k": 20,
}

In [ ]:
NOVA_MODEL_ID = "us.amazon.nova-lite-v1:0" # For this example we can use Nova Lite or Nova Pro models since structured output requires models that can efficiently make use of tools
CLAUDE_MODEL_ID = "us.anthropic.claude-3-haiku-20240307-v1:0"

bedrock_llm_nova_structured = ChatBedrock(
    model_id=NOVA_MODEL_ID,
    model_kwargs=INFORMATION_EXTRACTION_MODEL_PARAMETERS_WITH_STRUCTURED_OUTPUT,
    client=bedrock_runtime,
) # Langchain object to interact with Claude 3 models through Bedrock

bedrock_llm_haiku3_structured = ChatBedrock(
    model_id=CLAUDE_MODEL_ID,
    model_kwargs=INFORMATION_EXTRACTION_MODEL_PARAMETERS_WITH_STRUCTURED_OUTPUT,
    client=bedrock_runtime,
) # Langchain object to interact with Claude 3 models through Bedrock

In [ ]:
INFORMATION_EXTRACTION_PROMPT_SELECTOR = get_information_extraction_prompt_selector("es")

### Amazon Nova

In [ ]:
nova_information_extraction_prompt_template = INFORMATION_EXTRACTION_PROMPT_SELECTOR.get_prompt(NOVA_MODEL_ID)

structured_llm_nova = bedrock_llm_nova_structured.with_structured_output(InformationExtraction)

In [ ]:
structured_chain_nova = nova_information_extraction_prompt_template | structured_llm_nova

In [ ]:
start_time = time.time()
extracted_structured_information_nova = structured_chain_nova.invoke({
    "json_schema":InformacionRecibo.model_json_schema(), 
    "image_path":dataurl,
    "detail_parameter":"high"
})
end_time = time.time()

In [ ]:
extracted_structured_information_nova

In [ ]:
extracted_structured_information_nova.extracted_information

In [ ]:
print(f"Inference time: {nova_completion.response_metadata['metrics']['latencyMs'][0]} miliseconds")
print(f"Execution time {end_time - start_time} seconds")
print(f"Input tokens: {nova_completion.usage_metadata['input_tokens']}")
print(f"Input tokens: {nova_completion.usage_metadata['output_tokens']}")

### Anthropic Claude 3

In [ ]:
claude_information_extraction_prompt_template = INFORMATION_EXTRACTION_PROMPT_SELECTOR.get_prompt(CLAUDE_MODEL_ID)

structured_llm_haiku = bedrock_llm_haiku3_structured.with_structured_output(InformationExtraction)

In [ ]:
structured_chain_claude = claude_information_extraction_prompt_template | structured_llm_haiku

In [ ]:
start_time = time.time()
extracted_information_claude = structured_chain_claude.invoke({
    "json_schema":InformacionRecibo.model_json_schema(), 
    "image_path":dataurl,
    "detail_parameter":"high"
})
end_time = time.time()

In [ ]:
extracted_information_claude.extracted_information

In [ ]:
print(f"Execution time {end_time - start_time} seconds")

## Use Amazon Nova as a judge for the extraction task

In this section we will use Amazon Nova Pro as a judge to evaluate the extractions. This is a technique known as [LLM-as-a-judge](https://www.evidentlyai.com/llm-guide/llm-as-a-judge). We will ask the LLM to evaluate the extraction against human extracted information.

In [ ]:
LLM_AS_JUDGE_MODEL_PARAMETERS = {
    "max_tokens": 1000,
    "temperature": 0, # Low temperature since we want to extract data
    "top_k": 20,
}

In [ ]:
NOVA_MODEL_ID = "us.amazon.nova-pro-v1:0" # For this example we can use Nova Lite or Nova Pro models since structured output requires models that can efficiently make use of tools

bedrock_llm_as_judge_nova = ChatBedrock(
    model_id=NOVA_MODEL_ID,
    model_kwargs=LLM_AS_JUDGE_MODEL_PARAMETERS,
    client=bedrock_runtime,
) # Langchain object to interact with Claude 3 models through Bedrock

In [ ]:
llm_as_judge_system_prompt = """
You are an advanced evaluation system. Your will be presented with a JSON object with information extracted from a document in <extraction> and a ground truth 
human extracted information in <ground_truth>, in JSON format as well. Your task is to assign a score to the extraction in <extraction> based on how close the extracted information is 
to the information in <ground_truth>. You will provide a score in between 0 and 100, a higher score means the extracted information is closer to the ground 
truth. 

You will follow these rules for your task:

* You can reason about your evaluations. Use <scratchpad> to write your reasoning. Be very descriptive in your evaluation process.
* Do not add a preamble to your answer
* Write your response in <response>
* Your reference document is always the ground truth document. Start your analysis from it always.
* The scoring criteria is the same for every field in the JSON object, no matter its level.
* You will always assign a score from 0 to 3 points. 
* Evaluate only top fields. Assign a unique score to fields that are objects based on the criteria defined below.
* NEVER PENALIZE for capitalization or punctutation errors or extra information extracted.

To compare the extraction to the ground truth follow these steps:

1. Determine the number of top fieds within the ground truth JSON object, consider nested objects as a single field. For instance a field named products may be made up of a 
list of several product objects nevertheless you will assign it a global score. Write the number of fields in <number_fields>
2. Determine the maximum number of points that can be assigned. That is: 3*<number_fields>. Place this value in <max_points>
3. For each field you will assign a score based on the criteria that you will be given.
4. After assigning the points to each field add the total number of points obtained by the extraction and place the number in <total_points>
5. Normalize the results between 0 and 100 by computing (<total_points>/<max_points>)*100 and place them in <score>


Use the following criteria to assign a score to the extraction.

* 0 points if the information extracted is inconsistent with the ground truth data
* 1 point if the information is consistent but incomplete.
* 2 points if the information is complete but has one or more of the following errors:
    - It has typos
    - It is numerically different
* 3 points if the information is consistente, complete and has no evident errors.

Your response is should always contain the following fields:
    * <number_of_fields>
    * <max_points>
    * <total_points>
    * <score>
    
"""

llm_as_judge_user_prompt = """
Evaluate the followin extraction

<extraction>
{extraction}
</extraction>

with respect to

<ground_truth>
{ground_truth}
</ground_truth>

remember to start evaluating from the information in the ground truth.


"""

In [ ]:
system_prompt = SystemMessagePromptTemplate.from_template(
    llm_as_judge_system_prompt,
    input_variables=[],
    validate_template=True
)

user_prompt = HumanMessagePromptTemplate.from_template(
    llm_as_judge_user_prompt,
    input_variables=["extraction", "ground_truth"],
    validate_template=True
)

ll_as_judge_prompt_template = ChatPromptTemplate.from_messages([
    system_prompt,
    user_prompt,
])

In [ ]:
langchain_evaluation_nova = ll_as_judge_prompt_template | bedrock_llm_as_judge_nova

In [ ]:
extraction_evaluation_nova = langchain_evaluation_nova.invoke({
    "extraction": extracted_structured_information_nova.extracted_information,
    "ground_truth": json.dumps(ground_truth_receipt_information)
})

As we can see. Chain of thought is very useful for making the LLM reason through the evauation process, nevertheless it still struggles with the nested objects (like the product list). We may benefit a lot from the use of specialized tools to provide accurate scoring based on the type of data but we leave that as future work.

In [ ]:
extraction_evaluation_nova.content